In [1]:
import pandas as pd
from mftool import Mftool
from datetime import datetime, timedelta
import os
from openpyxl import load_workbook

# Function to calculate returns for different time frames starting from a specific date
def calculate_returns1(latest_nav, nav_series, n):
    return (((latest_nav / nav_series.shift(n))) - 1)*100

# Function to calculate returns for different time frames starting from a specific date
def calculate_returns(latest_nav, nav_series, n):
    return (((latest_nav / nav_series.shift(n)) ** (1 / n)) - 1)*100

# Initialize Mftool
mf = Mftool()

# List of scheme codes
#scheme_codes_list = ["125595", "103168", "101738"]  # Add all the scheme codes
# List of scheme codes as strings
scheme_codes_list = [
    "125595", "103168", "101738", "100033", "103166", "103309", "103174", "103111", "101592", "108167",
    "105804", "112277", "145112", "114564", "149382", "125350", "102020", "102920", "105875", "104481",
    "105989", "101635", "100471", "102883", "100520", "105817", "100473", "103360", "101764", "101762",
    "102760", "130496", "105758", "130502", "102000", "102252", "146771", "151034", "151133", "109445",
    "108466", "129310", "111957", "100349", "102528", "101228", "143873", "106823", "117620", "102594",
    "114458", "103234", "112090", "149182", "102875", "148621", "112932", "147203", "114931", "143785",
    "107578", "147479", "122387", "147701", "127039", "101862", "102751", "104637", "100377", "106235",
    "101161", "113177", "103085", "100380", "122640", "125305", "133858", "103504", "102414", "102756",
    "103145", "102941", "149886", "125494", "109059", "150568", "149532", "105001", "101539", "100795",
    "135794", "135797", "101672", "135804", "101824", "100475", "102328", "117312", "100477", "150347"
]

# Manual input of start date
start_date = "01-01-2006"

# Convert start_date to datetime.date object
start_date_obj = pd.to_datetime(start_date, format="%d-%m-%Y")

# Create a list to store the results for all schemes
all_results = []

# Loop through each scheme code
for scheme_code in scheme_codes_list:
    # Check if the input scheme code exists in the list of scheme codes
    scheme_codes = mf.get_scheme_codes()
    if scheme_code not in scheme_codes:
        print(f"Invalid Scheme Code '{scheme_code}'. Please enter a valid scheme code.")
    else:
        # Fetch historical NAV data for the given scheme code and date range
        nav_data = mf.get_scheme_historical_nav(scheme_code, start_date_obj, datetime.today().date())

        # Extract the actual NAV data from the dictionary
        nav_data = nav_data['data']

        # Create a DataFrame from the NAV data
        nav_df = pd.DataFrame(nav_data)

        # Convert the "nav" column to numeric values
        nav_df["nav"] = pd.to_numeric(nav_df["nav"], errors="coerce")

        # Filter out any missing or zero NAV values
        nav_df = nav_df.dropna(subset=["nav"]).query("nav != 0")

        # Convert the "date" column to datetime objects
        nav_df["date"] = pd.to_datetime(nav_df["date"], format="%d-%m-%Y")

        # Sort the NAV data by date in ascending order
        nav_df.sort_values("date", inplace=True)

        # Convert today's date to a datetime object
        today_date = datetime.today()

        # Calculate returns for different time frames starting from each date
        nav_df["1 Year Return"] = calculate_returns1(nav_df["nav"], nav_df["nav"], 249)
        nav_df["2 Year Return"] = calculate_returns(nav_df["nav"], nav_df["nav"].shift(250 * 2 - 1), 2)
        nav_df["3 Year Return"] = calculate_returns(nav_df["nav"], nav_df["nav"].shift(250 * 3 - 1), 3)
        nav_df["5 Year Return"] = calculate_returns(nav_df["nav"], nav_df["nav"].shift(250 * 5 - 1), 5)
        nav_df["10 Year Return"] = calculate_returns(nav_df["nav"], nav_df["nav"].shift(250 * 10 - 1), 10)

        # Filter the data to include only rows within the specified date range
        nav_df = nav_df[(nav_df["date"] >= start_date_obj) & (nav_df["date"] <= datetime.today())]

        # Retrieve the scheme name from the scheme_codes dictionary
        scheme_name = scheme_codes[scheme_code]

        # Add the scheme name and DataFrame to the results list
        all_results.append({"Scheme Code": scheme_code, "Scheme Name": scheme_name, "Data": nav_df})

# Loop through all the results and save each DataFrame to an Excel file

# Create a Pandas ExcelWriter object to save all data in one file
output_file = os.path.join("E:/mftool/", "all_schemes_returns.xlsx")
excel_writer = pd.ExcelWriter(output_file, engine='xlsxwriter')


# Loop through all the results and save each DataFrame to a separate sheet
for result in all_results:
    scheme_code = result["Scheme Code"]
    scheme_name = result["Scheme Name"]
    nav_df = result["Data"]

    # Remove any forward slashes from the scheme_name to avoid filename issues
    scheme_name = scheme_name.replace("/", "-")

    # Generate a unique identifier for each scheme based on scheme_code and the first few characters of the scheme_name
    unique_identifier = f"{scheme_code}_{scheme_name[:15]}"

    # Save the DataFrame to the ExcelWriter with the unique_identifier as the sheet name
    nav_df.to_excel(excel_writer, sheet_name=unique_identifier, index=False)
    
    # Get the sheet object
    sheet = excel_writer.sheets[unique_identifier]

    # Write scheme name and code as a title in the sheet
    sheet.write('H1', f"Scheme Name: {scheme_name} (Code: {scheme_code})")

# Save the ExcelWriter object to the output file
excel_writer.save()

print(f"All data saved to {output_file}")

Invalid Scheme Code '147701'. Please enter a valid scheme code.
All data saved to E:/mftool/all_schemes_returns.xlsx
